In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import sys
import os
import importlib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datastory import DataStory

from google.cloud import secretmanager

sys.path.append('../lib')
import pandas_utils
import pesys_utils
import autograd_convenience
import utils

importlib.reload(pandas_utils)
importlib.reload(pesys_utils)
importlib.reload(autograd_convenience)
importlib.reload(utils)

utils.set_secrets_as_env()

In [ ]:
try:
    con = pesys_utils.open_pen_connection()
except cx_Oracle.DatabaseError:
    con.close()
    con = pesys_utils.open_pen_connection()
    
df_list = pandas_utils.pandas_from_sql('../sql/forslag_auto.sql', con, tuning = 100000)
#df_list = pandas_utils.pandas_from_sql('../sql/v8_kun_norge.sql', con)
con.close()
#df_list["ÅR-TERTIAL"] = df_list["ÅR-TERTIAL"].apply(lambda x: x[:4] + "-" + x[4:])

In [ ]:
df_list.head()

In [ ]:
df_list["DATO_VEDTAK"] = df_list.DATO_VEDTAK.apply(pd.Timestamp.to_pydatetime)
df_list["AAR_TERTIAL"] = df_list.DATO_VEDTAK.apply(lambda x: str(x.year) + "-" + str((x.month - 1) // 4 + 1))

In [ ]:
df_list.head()

In [ ]:
from google.cloud.bigquery import Client, LoadJobConfig

table_id = f'pensjon-saksbehandli-prod-1f83.vedtak.vedtak_automatisering_v2'
job_config = LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client = Client(project="pensjon-saksbehandli-prod-1f83")

job = client.load_table_from_dataframe(df_list, table_id, job_config=job_config)
job.result()

# Krav

In [ ]:
try:
    con = pesys_utils.open_pen_connection()
except cx_Oracle.DatabaseError:
    con.close()
    con = pesys_utils.open_pen_connection()
    
df_krav = pandas_utils.pandas_from_sql('../sql/forslag_auto_krav.sql', con, tuning=100000, lowercase=True)
#df_list = pandas_utils.pandas_from_sql('../sql/v8_kun_norge.sql', con)
con.close()
#df_list["ÅR-TERTIAL"] = df_list["ÅR-TERTIAL"].apply(lambda x: x[:4] + "-" + x[4:])

In [ ]:
def add_time_columns_to_df(df, tcol):
    df["tertial"] = (df[tcol].dt.month - 1) // 4 + 1
    df["year"] = df[tcol].dt.year
    df["month"] = df[tcol].dt.month
    df["0month"] = df["month"].apply(str).apply(lambda x: x.zfill(2))
    df["year_month"] = df[["year", "0month"]].apply(lambda row: "-".join(row.values.astype(str)), axis=1)
    df["year_tertial"] = df[["year", "tertial"]].apply(lambda row: "-".join(row.values.astype(str)), axis=1)
    return df

df_timed = add_time_columns_to_df(df_krav, "dato_opprettet")

In [ ]:
df_krav.head()

In [ ]:
df_forstegang = df_krav[(df_krav.sakstype == "Alderspensjon") & (df_forstegang.batch == "Ikke batch") & (df_forstegang.kravtype.isin(pesys_utils.forstegang))]

In [ ]:
def group_df_on_var(df, period, var):
    if type(var) is list or type(var) is tuple:
        df = df.groupby([period] + var)[period].count().reset_index(name="antall")
    else:
        df = df.groupby([period, var])[var].count().reset_index(name="antall")

    df = df.merge(df.groupby([period], as_index=False).antall.sum(),
                    left_on=[period],
                    right_on=[period],
                    suffixes=(""," totalt")
            )
    df["andel"] = df.antall.divide(df["antall totalt"])
    return df

In [ ]:
df_auto = group_df_on_var(df_forstegang, "year_month", "automatisering")

In [ ]:
df_auto.head()

In [ ]:
px.line(df_auto[df_auto.automatisering == "AUTO"], "year_month", "andel")